### Curriculum Modeling: CNN Evaluation

In this notebook, we conduct the performance evaluation for the neural networks trained with both interleaved and blocked (control) training schedules. 

Taining effect, both overall and by training condition (interleaved vs blocked), on accuracy, sensitivity, and specificity was assessed using mixed effects log binomial regression models. Accuracy, sensitivity, and specificity for each training condition were calculated using marginal means estimations and a comparison between pre- and post-test performance was assessed using Wald tests.

In this notebook, we will use the package lme4 to create our mixed effects regression models, and the package emmeans for marginal mean estimations.

To get started, we load the required packages:

In [2]:
require(reshape2)
require(lme4)
require(compiler)
require(parallel)
require(boot)
require(lattice)

In [4]:
# load cnn results (generated in Radiology_Curriculum_Modeling.ipynb)
data <- read.csv('../reports/cnn_results.csv')
data

Condition,Fold,Time,Abnormality,Correct,ID,correct
nonblocked,fold_0,Post,Yes,Yes,echo,1
nonblocked,fold_0,Post,Yes,Yes,bravo,1
nonblocked,fold_0,Post,No,Yes,victor,1
nonblocked,fold_0,Post,No,No,echo,0
nonblocked,fold_0,Post,Yes,No,victor,0
nonblocked,fold_0,Post,Yes,Yes,hulu,1
nonblocked,fold_0,Post,No,No,charlie,0
nonblocked,fold_0,Post,Yes,Yes,echo,1
nonblocked,fold_0,Post,Yes,Yes,echo,1
nonblocked,fold_0,Post,No,No,bravo,0


#### 1.0 Mixed Effects Log Binomial Model

In the following cells, we create a mixed effects log-binomial regression model of the form Correct ~ Time * Condition * Abnormality. The fold is added as a random effect. 

In [5]:
# depending on which condition we want to investigate, we can change the factor levels of a factor column
data$Time <- factor(data$Time, levels = c("Pre", "Post"))
data$Abnormality <- factor(data$Abnormality, levels = c("Yes", "No"))
levels(data$Time)
levels(data$Abnormality)

[1] "Pre"  "Post"

[1] "Yes" "No"

In [7]:
# estimate the model and store results in m
m <- glmer(correct ~ Time * Condition * Abnormality + (1|Fold)
           , data = data, family=poisson(link="log"),
    nAGQ = 0)

# print the mod results without correlations among fixed effects
print(m, corr = TRUE)

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 0) [glmerMod]
 Family: poisson  ( log )
Formula: correct ~ Time * Condition * Abnormality + (1 | Fold)
   Data: data
      AIC       BIC    logLik  deviance  df.resid 
1263.2662 1305.4164 -622.6331 1245.2662       790 
Random effects:
 Groups Name        Std.Dev.
 Fold   (Intercept) 0.1112  
Number of obs: 799, groups:  Fold, 10
Fixed Effects:
                               (Intercept)  
                                   -1.6374  
                                  TimePost  
                                    1.4702  
                       Conditionnonblocked  
                                    0.9139  
                             AbnormalityNo  
                                    0.2095  
              TimePost:Conditionnonblocked  
                                   -1.1555  
                    TimePost:AbnormalityNo  
                                   -0.9106  
         Con

In [8]:
se <- sqrt(diag(vcov(m)))

# table of estimates with 95% CI
tab <- cbind(Est = fixef(m), LL = fixef(m) - 1.96 * se, UL = fixef(m) + 1.96 *
    se)

# calculate risk ratios by taking the exponential 
exp(tab)

,Est,LL,UL
(Intercept),0.1944837,0.13342598,0.2834824
TimePost,4.3500427,2.90607213,6.5114940
Conditionnonblocked,2.4941385,1.58744346,3.9187076
AbnormalityNo,1.2330830,0.62924237,2.4163878
TimePost:Conditionnonblocked,0.3149079,0.19028908,0.5211388
TimePost:AbnormalityNo,0.4022949,0.17790163,0.9097229
Conditionnonblocked:AbnormalityNo,0.1999665,0.06857339,0.5831216
TimePost:Conditionnonblocked:AbnormalityNo,9.0729462,2.63890453,31.1941381


In [13]:
# Calculate estimates and confidence intervals
ci <- confint(m, method='Wald')[-1,]
est <- fixef(m)
exp_est <- exp(est)
exp_ci <- exp(ci)

# Get p-values
p <- summary(m)$coefficients[,4]

# Combine results into one dataframe
results <- data.frame(RR = exp_est,
                      ci_low = exp_ci[,1],
                      ci_high = exp_ci[,2],
                      p_value = p)

# Print results
results

,RR,ci_low,ci_high,p_value
(Intercept),0.1944837,0.13342690,0.2834804,1.633687e-17
TimePost,4.3500427,2.90609367,6.5114458,9.099132e-13
Conditionnonblocked,2.4941385,1.58745664,3.9186751,7.348555e-05
AbnormalityNo,1.2330830,0.62925015,2.4163579,5.415930e-01
TimePost:Conditionnonblocked,0.3149079,0.19029084,0.5211339,6.928653e-06
TimePost:AbnormalityNo,0.4022949,0.17790430,0.9097093,2.872248e-02
Conditionnonblocked:AbnormalityNo,0.1999665,0.06857473,0.5831101,3.200864e-03
TimePost:Conditionnonblocked:AbnormalityNo,9.0729462,2.63896441,31.1934302,4.650940e-04


In [21]:
library(emmeans)

# Calculate the EMMs and pairwise comparisons
emm.1 <- emmeans(m, ~ Time)
pairwise <- as.data.frame(pairs(regrid(emm.1), transform = "log"))

# print the EMM - corresponding to Pre and Post-test accuracy in this case
summary(emm.1, type="response")

# Print the p.value
pairwise

NOTE: Results may be misleading due to involvement in interactions


Time,rate,SE,df,asymp.LCL,asymp.UCL
Pre,0.2280757,0.03215383,Inf,0.1730126,0.3006633
Post,0.6128775,0.05277925,Inf,0.5176910,0.7255656


contrast,estimate,SE,df,z.ratio,p.value
Pre - Post,-0.3848018,0.05893146,Inf,-6.52965,6.592364e-11


In [23]:
# Calculate the EMMs and pairwise comparisons
emm.1 <- emmeans(m, ~ Condition * Time)
pairwise <- as.data.frame(pairs(regrid(emm.1), transform = "log"))

# print the EMM - corresponding to Pre and Post-test accuracy by training schedule
summary(emm.1, type="response")

# Print the p.value
pairwise

NOTE: Results may be misleading due to involvement in interactions


Condition,Time,rate,SE,df,asymp.LCL,asymp.UCL
blocked,Pre,0.2159630,0.03859005,Inf,0.1521527,0.3065343
nonblocked,Pre,0.2408677,0.05253489,Inf,0.1570818,0.3693444
blocked,Post,0.5958613,0.07618814,Inf,0.4637764,0.7655643
nonblocked,Post,0.6303796,0.07274236,Inf,0.5027802,0.7903622


contrast,estimate,SE,df,z.ratio,p.value
"blocked,Pre - nonblocked,Pre",-0.02490471,0.06518517,Inf,-0.3820610,9.810299e-01
"blocked,Pre - blocked,Post",-0.37989826,0.08158859,Inf,-4.6562670,1.911824e-05
"blocked,Pre - nonblocked,Post",-0.41441664,0.08234465,Inf,-5.0327085,2.886046e-06
"nonblocked,Pre - blocked,Post",-0.35499354,0.09254485,Inf,-3.8359083,7.220718e-04
"nonblocked,Pre - nonblocked,Post",-0.38951192,0.08542338,Inf,-4.5597812,3.034700e-05
"blocked,Post - nonblocked,Post",-0.03451838,0.10533795,Inf,-0.3276918,9.878580e-01


In [25]:
# Calculate the EMMs and pairwise comparisons
emm.1 <- emmeans(m, ~ Condition * Time * Abnormality)
pairwise <- as.data.frame(pairs(regrid(emm.1), transform = "log"))

# print the EMM - corresponding to Pre and Post-test sensitivity (abnormality YES) and specificity (abnormality NO)
# by training schedule (blocked vs non-blocked)
summary(emm.1, type="response")

# Print the p.value
pairwise

Condition,Time,Abnormality,rate,SE,df,asymp.LCL,asymp.UCL
blocked,Pre,Yes,0.1944837,0.03738865,Inf,0.13342690,0.2834804
nonblocked,Pre,Yes,0.4850693,0.06170287,Inf,0.37803095,0.6224153
blocked,Post,Yes,0.8460124,0.08606708,Inf,0.69307744,1.0326942
nonblocked,Post,Yes,0.6644785,0.07423935,Inf,0.53380152,0.8271458
blocked,Pre,No,0.2398145,0.07024906,Inf,0.13506176,0.4258127
nonblocked,Pre,No,0.1196061,0.04918913,Inf,0.05341815,0.2678046
blocked,Post,No,0.4196755,0.09393078,Inf,0.27064477,0.6507700
nonblocked,Post,No,0.5980306,0.11317440,Inf,0.41270304,0.8665811


contrast,estimate,SE,df,z.ratio,p.value
"blocked,Pre,Yes - nonblocked,Pre,Yes",-0.29058559,0.07214676,Inf,-4.0277012,1.454363e-03
"blocked,Pre,Yes - blocked,Post,Yes",-0.65152872,0.08939141,Inf,-7.2884939,8.829049e-12
"blocked,Pre,Yes - nonblocked,Post,Yes",-0.46999479,0.08312276,Inf,-5.6542251,4.362658e-07
"blocked,Pre,Yes - blocked,Pre,No",-0.04533084,0.07811481,Inf,-0.5803105,9.991053e-01
"blocked,Pre,Yes - nonblocked,Pre,No",0.07487758,0.06178578,Inf,1.2118902,9.287181e-01
"blocked,Pre,Yes - blocked,Post,No",-0.22519175,0.09907974,Inf,-2.2728334,3.086481e-01
"blocked,Pre,Yes - nonblocked,Post,No",-0.40354694,0.11919042,Inf,-3.3857330,1.629427e-02
"nonblocked,Pre,Yes - blocked,Post,Yes",-0.36094313,0.10589988,Inf,-3.4083430,1.508959e-02
"nonblocked,Pre,Yes - nonblocked,Post,Yes",-0.17940919,0.08785046,Inf,-2.0422111,4.531302e-01
"nonblocked,Pre,Yes - blocked,Pre,No",0.24525475,0.09349959,Inf,2.6230569,1.475114e-01
